# GMM 测试


## 导入数据 & 数据分析


In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV, GroupKFold, KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 设置中文字体
plt.rcParams["font.family"] = "SimHei"  # 黑体 SimHei 支持中文
plt.rcParams["axes.unicode_minus"] = False  # 正常显示负号

## 函数：解析表头 & 导入数据


In [ ]:
def identify_attributes(file_path):
    """
    从Petrel格式的地震属性文件中识别属性

    参数:
        file_path (str): Petrel文件的路径

    返回:
        tuple: (属性列表, END ATTRIBUTES所在行号)
    """
    print(f"正在识别文件属性: {file_path}")

    # 读取文件内容
    with open(file_path, "r") as file:
        lines = file.readlines()

    attr_start = -1
    attr_end = -1
    attributes = []

    # 查找属性部分的起始和结束位置
    for i, line in enumerate(lines):
        if line.strip() == "ATTRIBUTES":
            attr_start = i
        elif line.strip() == "END ATTRIBUTES":
            attr_end = i
            break

    print(f"识别到 END ATTRIBUTES 位于第 {attr_end} 行")

    # 提取属性名称
    if attr_start != -1 and attr_end != -1:
        # 跳过ATTRIBUTES行和下一行的1,N
        for i in range(attr_start + 2, attr_end):
            line = lines[i].strip()
            # 直接查找最后一个英文逗号，之后的内容即为属性名
            last_comma_index = line.rfind(",")
            if last_comma_index != -1:
                attr_name = line[last_comma_index + 1 :].strip()
                attributes.append(attr_name)

    print(f"识别到 {len(attributes)} 个属性:")
    for attr in attributes:
        print(f"  - {attr}")

    return attributes, attr_end


def parse_petrel_file(file_path):
    """
    解析Petrel格式的地震属性文件

    参数:
        file_path (str): Petrel文件的路径

    返回:
        pandas.DataFrame: 包含解析后数据的DataFrame
    """
    print(f"正在解析文件: {file_path}")

    # 步骤1: 识别属性和确定跳过的行数
    attributes, skip_rows = identify_attributes(file_path)

    # 读取文件内容以分析数据结构
    with open(file_path, "r") as file:
        lines = file.readlines()

    # 步骤2: 解析数据列数
    if skip_rows + 1 < len(lines):
        first_data_line = lines[skip_rows + 1].strip()
        data_columns = first_data_line.split()
        num_columns = len(data_columns)
        print(f"解析到数据有 {num_columns} 列")
    else:
        print("未找到数据行")
        return None

    # 步骤3: 创建列名
    column_names = ["X", "Y", "Z"]

    # 添加中间的占位符列名
    placeholder_count = num_columns - 3 - len(attributes)
    for i in range(placeholder_count):
        column_names.append(f"placeholder{i + 1}")

    # 添加属性列名
    column_names.extend(attributes)

    print(f"总列数: {num_columns}, 其中:")
    print(f"  - 3 列为坐标 (X, Y, Z)")
    print(f"  - {placeholder_count} 列为占位符")
    print(f"  - {len(attributes)} 列为属性")

    # 读取数据
    try:
        df = pd.read_csv(
            file_path,
            delim_whitespace=True,
            skiprows=skip_rows + 1,  # 跳过END ATTRIBUTES行及之前的所有行
            names=column_names,
            dtype=float,
            engine="python",
        )
        print(f"成功读取数据，共 {len(df)} 行")
        return df
    except Exception as e:
        print(f"读取数据时出错: {str(e)}")
        return None

In [ ]:
H6_2_attr, _ = identify_attributes("../data/H6-2_attr")

data_H6_2_attr = parse_petrel_file("../data/H6-2_attr")

## 解析井震数据

使用 xlsx / csv 数据，注意表名为 Sheet1，注意数据需包含表头

请检查表头和下面代码中的 selected_columns 是否一致


In [ ]:
file_H6_2_well = "../data/H6-2_well.xlsx"
data_H6_2_well = pd.read_excel(file_H6_2_well, sheet_name="Sheet1")

selected_columns = [
    "X",
    "Y",
    "Z",
    "Well",
    "Thickness of LITHOLOGIES(1: sand)",
    # "LITHOLOGIES(1: sand)",
    # "标签",
    *H6_2_attr,
]

# 只选择 Surface 为 H6-2 的行，并丢弃砂厚为 NaN 的行
data_H6_2_well_selected = (
    data_H6_2_well.loc[data_H6_2_well["Surface"] == "H6-2", selected_columns]
    .replace(-999, np.nan)
    .dropna(subset=["Thickness of LITHOLOGIES(1: sand)"])
    .reset_index(drop=True)
)

In [ ]:
data_H6_2_well_selected

## 函数：缺失值处理、异常值替换


In [ ]:
def preprocess_features(data, attribute_columns, missing_values=[-999], verbose=True):
    """
    预处理特征数据，包括缺失值处理、异常值替换和特征筛选

    参数:
        data (DataFrame): 包含特征的数据框
        attribute_columns (list): 需要处理的特征列名列表
        missing_values (list): 要替换为NaN的值列表，默认为[-999]
        verbose (bool): 是否打印详细信息，默认为True

    返回:
        DataFrame: 处理后的特征数据框
    """
    # 提取特征
    features = data[attribute_columns].copy()

    # 替换缺失值
    for val in missing_values:
        features = features.replace(val, np.nan)

    if verbose:
        print(f"处理前特征: {features.shape}")

    # 检查缺失值情况
    missing_per_column = features.isna().sum()
    if verbose:
        print("\n每列缺失值数量:")
    missing_cols = []
    for col, missing in missing_per_column.items():
        missing_ratio = missing / len(features) * 100
        if verbose:
            print(f"  - {col}: {missing} ({missing_ratio:.2f}%)")

        # 标记缺失率较高的列
        if missing_ratio >= 89.9:
            missing_cols.append(col)

    if missing_cols:
        if verbose:
            print(f"\n删除以下全部缺失的列: {missing_cols}")
        features = features.drop(columns=missing_cols)

    # 填充剩余列中的NaN值
    # 对于每一列，如果均值是NaN，则使用0填充
    for col in features.columns:
        if pd.isna(features[col].mean()):
            if verbose:
                print(f"列 '{col}' 的均值为NaN，使用0填充")
            features[col] = features[col].fillna(0)
        else:
            features[col] = features[col].fillna(features[col].mean())

    if verbose:
        print(f"\n清理并填充后的特征形状: {features.shape}")

    # 检查是否仍有NaN值
    if features.isna().any().any():
        if verbose:
            print("警告：数据中仍然存在NaN值，将它们替换为0")
        features = features.fillna(0)

    return features

## 函数：PCA 降维

会失去可解释性，这是一次尝试。


In [ ]:
def perform_pca_analysis(
    data,
    attribute_columns,
    variance_threshold=0.95,
    output_dir="output",
    missing_values=[-999],
):
    """
    对地震属性数据进行PCA降维分析

    参数:
        data (DataFrame): 包含地震属性的数据框
        attribute_columns (list): 需要进行分析的属性列名列表
        variance_threshold (float): PCA保留方差的阈值，默认为0.95
        output_dir (str): 输出图表的目录，默认为"output"
        missing_values (list): 要替换为NaN的值列表，默认为[-999]

    返回:
        dict: 包含PCA分析结果的字典
    """
    print("======== PCA降维分析开始 ========")
    print(f"数据集大小: {data.shape}")

    # 预处理特征
    features = preprocess_features(data, attribute_columns, missing_values)

    # 提取对应的坐标
    coords_clean = data[["X", "Y", "Z"]]

    # 标准化
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    # 应用PCA
    pca = PCA()
    pca_result = pca.fit_transform(features_scaled)

    # 计算累积解释方差比
    explained_variance_ratio_cumsum = np.cumsum(pca.explained_variance_ratio_)

    # 确定需要保留的主成分数量
    n_components = np.argmax(explained_variance_ratio_cumsum >= variance_threshold) + 1
    print(f"为保留至少{variance_threshold * 100}%的方差，需要保留{n_components}个主成分")

    # 绘制解释方差比
    plt.figure(figsize=(10, 6))
    plt.bar(
        range(1, len(pca.explained_variance_ratio_) + 1),
        pca.explained_variance_ratio_,
        alpha=0.8,
        label="单个方差贡献",
    )
    plt.step(
        range(1, len(pca.explained_variance_ratio_) + 1),
        explained_variance_ratio_cumsum,
        where="mid",
        label="累积方差贡献",
    )
    plt.axhline(
        y=variance_threshold,
        linestyle="--",
        color="r",
        label=f"{variance_threshold * 100}%方差阈值",
    )
    plt.axvline(
        x=n_components,
        linestyle="--",
        color="g",
        label=f"选择的主成分数({n_components})",
    )
    plt.title("PCA解释方差比")
    plt.xlabel("主成分数量")
    plt.ylabel("解释方差比")
    plt.legend()
    plt.grid(True)
    plt.savefig(
        os.path.join(output_dir, "pca_explained_variance.png"),
        dpi=300,
        bbox_inches="tight",
    )
    plt.show()

    # 使用选择的主成分数重新应用PCA
    pca = PCA(n_components=n_components)
    features_pca = pca.fit_transform(features_scaled)
    print(f"降维后的特征形状: {features_pca.shape}")

    # 打印每个主成分对应的原始特征贡献
    component_contributions = pd.DataFrame(
        pca.components_,
        columns=features.columns,  # 注意这里使用的是已清理的列
        index=[f"PC{i + 1}" for i in range(n_components)],
    )
    print("\n主成分与原始特征的关系:")
    print(component_contributions)

    print("======== PCA降维分析完成 ========")

    # 返回结果
    return {
        "pca": pca,
        "scaler": scaler,
        "n_components": n_components,
        "component_contributions": component_contributions,
        "features_clean": features,
        "features_pca": features_pca,
        "coords_clean": coords_clean,
    }

In [ ]:
# 对地震属性数据进行PCA降维
pca_results = perform_pca_analysis(
    data=data_H6_2_attr,
    attribute_columns=H6_2_attr,
    variance_threshold=0.95,
    output_dir=output_dir,
)

## 函数：GMM (Gaussian Mixture Model)


In [ ]:
def evaluate_gmm_clusters(features_pca, max_clusters=10, output_dir="output"):
    """
    对不同聚类数的GMM模型进行评估，绘制BIC和AIC曲线

    参数:
        features_pca (ndarray): PCA降维后的特征矩阵
        max_clusters (int): 最大聚类数量，默认为10
        output_dir (str): 输出图表的目录，默认为"output"

    返回:
        dict: 包含评估结果的字典
    """
    print("======== GMM聚类数评估开始 ========")

    # 确定最佳GMM聚类数
    n_components_range = range(1, max_clusters + 1)
    models = []
    bic_scores = []
    aic_scores = []

    for n_comp in n_components_range:
        # 训练GMM模型
        gmm = GaussianMixture(
            n_components=n_comp,
            covariance_type="full",
            random_state=42,
            n_init=10,  # 多次初始化以获得更稳定的结果
        )
        gmm.fit(features_pca)

        # 保存模型和得分
        models.append(gmm)
        bic_scores.append(gmm.bic(features_pca))
        aic_scores.append(gmm.aic(features_pca))
        print(f"聚类数量 {n_comp}: BIC = {bic_scores[-1]:.2f}, AIC = {aic_scores[-1]:.2f}")

    # 绘制BIC和AIC曲线
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(n_components_range, bic_scores, "o-", label="BIC")
    plt.axvline(
        np.argmin(bic_scores) + 1,
        linestyle="--",
        color="r",
        label=f"最佳聚类数 = {np.argmin(bic_scores) + 1}",
    )
    plt.xlabel("聚类数量")
    plt.ylabel("BIC分数")
    plt.title("BIC评分曲线")
    plt.grid(True)
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(n_components_range, aic_scores, "o-", label="AIC")
    plt.axvline(
        np.argmin(aic_scores) + 1,
        linestyle="--",
        color="r",
        label=f"最佳聚类数 = {np.argmin(aic_scores) + 1}",
    )
    plt.xlabel("聚类数量")
    plt.ylabel("AIC分数")
    plt.title("AIC评分曲线")
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "gmm_bic_aic.png"), dpi=300, bbox_inches="tight")
    plt.show()

    # 计算BIC和AIC的变化率
    if len(bic_scores) > 1:
        bic_changes = np.diff(bic_scores) / np.array(bic_scores[:-1])
        aic_changes = np.diff(aic_scores) / np.array(aic_scores[:-1])

        plt.figure(figsize=(12, 6))
        plt.plot(range(2, max_clusters + 1), bic_changes, "o-", label="BIC变化率")
        plt.plot(range(2, max_clusters + 1), aic_changes, "s-", label="AIC变化率")
        plt.axhline(y=0, linestyle="--", color="gray")
        plt.xlabel("聚类数量")
        plt.ylabel("相对变化率")
        plt.title("BIC和AIC相对变化率")
        plt.legend()
        plt.grid(True)
        plt.savefig(
            os.path.join(output_dir, "gmm_bic_aic_changes.png"),
            dpi=300,
            bbox_inches="tight",
        )
        plt.show()

        # 尝试找出变化率变化最大的点(可能的"肘部")
        if len(bic_changes) > 1:
            bic_elbow = np.argmax(np.abs(np.diff(bic_changes))) + 2
            aic_elbow = np.argmax(np.abs(np.diff(aic_changes))) + 2
            print(f"BIC变化率最大变化点: 聚类数 = {bic_elbow}")
            print(f"AIC变化率最大变化点: 聚类数 = {aic_elbow}")

    # 选择最佳模型（基于BIC）
    best_n_components = np.argmin(bic_scores) + 1
    best_gmm = models[best_n_components - 1]
    print(f"\n基于BIC的最佳聚类数: {best_n_components}")
    print(f"基于AIC的最佳聚类数: {np.argmin(aic_scores) + 1}")

    print("======== GMM聚类数评估完成 ========")

    # 返回结果
    return {
        "models": models,
        "bic_scores": bic_scores,
        "aic_scores": aic_scores,
        "best_n_components": best_n_components,
        "best_gmm": best_gmm,
    }

In [ ]:
# 首先评估最佳聚类数
# gmm_evaluation = evaluate_gmm_clusters(
#     features_pca=pca_results["features_pca"], max_clusters=10, output_dir=output_dir
# )

# # 使用不同的聚类数执行GMM聚类
# # 根据BIC/AIC结果选择的最佳聚类数
# best_n = gmm_evaluation["best_n_components"]

In [ ]:
def perform_gmm_clustering(features_pca, coords, n_clusters, output_dir="output", random_state=42, prefix=""):
    """
    执行指定聚类数的GMM聚类并可视化结果

    参数:
        features_pca (ndarray): PCA降维后的特征矩阵
        coords (DataFrame): 对应的坐标数据
        n_clusters (int): 聚类数量
        output_dir (str): 输出图表的目录，默认为"output"
        random_state (int): 随机种子，默认为42
        prefix (str): 输出文件名前缀，默认为""

    返回:
        dict: 包含GMM聚类结果的字典
    """
    print(f"======== 执行 {n_clusters} 聚类的GMM分析 ========")

    # 训练GMM模型
    gmm = GaussianMixture(
        n_components=n_clusters,
        covariance_type="full",
        random_state=random_state,
        n_init=10,  # 多次初始化以获得更稳定的结果
    )
    gmm.fit(features_pca)

    # 获取聚类标签和概率
    cluster_labels = gmm.predict(features_pca)
    cluster_probs = gmm.predict_proba(features_pca)

    # 将聚类结果添加到原始数据中
    result_df = pd.DataFrame(
        {
            "X": coords["X"],
            "Y": coords["Y"],
            "Z": coords["Z"],
            "Cluster": cluster_labels,
        }
    )

    # 文件名前缀
    file_prefix = f"{prefix}_{n_clusters}_clusters_" if prefix else f"{n_clusters}_clusters_"

    # 可视化1: 空间分布
    plt.figure(figsize=(12, 10))
    scatter = plt.scatter(
        result_df["X"],
        result_df["Y"],
        c=result_df["Cluster"],
        cmap="viridis",
        s=30,
        alpha=0.8,
        linewidths=0.5,
    )
    plt.colorbar(scatter, label="聚类")
    plt.title(f"样本点的聚类分布 (聚类数={n_clusters})")
    plt.xlabel("X坐标")
    plt.ylabel("Y坐标")
    plt.savefig(
        os.path.join(output_dir, f"{file_prefix}gmm_spatial.png"),
        dpi=300,
        bbox_inches="tight",
    )
    plt.show()

    # 可视化2: PCA投影
    if features_pca.shape[1] >= 2:
        plt.figure(figsize=(10, 8))
        scatter = plt.scatter(
            features_pca[:, 0],
            features_pca[:, 1],
            c=cluster_labels,
            cmap="viridis",
            s=30,
            alpha=0.7,
        )
        plt.colorbar(scatter, label="聚类")
        plt.title(f"PCA投影下的聚类分布 (聚类数={n_clusters})")
        plt.xlabel("主成分1")
        plt.ylabel("主成分2")
        plt.savefig(
            os.path.join(output_dir, f"{file_prefix}gmm_pca_projection.png"),
            dpi=300,
            bbox_inches="tight",
        )
        plt.show()

    # 统计各聚类的样本数
    cluster_counts = result_df["Cluster"].value_counts().sort_index()

    plt.figure(figsize=(10, 6))
    cluster_counts.plot(kind="bar")
    plt.title(f"各聚类样本数量分布 (聚类数={n_clusters})")
    plt.xlabel("聚类")
    plt.ylabel("样本数量")
    plt.grid(axis="y")
    plt.savefig(
        os.path.join(output_dir, f"{file_prefix}gmm_cluster_counts.png"),
        dpi=300,
        bbox_inches="tight",
    )
    plt.show()

    print("\n各聚类样本数量:")
    for cluster, count in cluster_counts.items():
        print(f"聚类 {cluster}: {count} 样本 ({count / len(result_df) * 100:.2f}%)")

    print(f"======== {n_clusters} 聚类的GMM分析完成 ========")

    # 返回结果
    return {
        "gmm": gmm,
        "cluster_labels": cluster_labels,
        "cluster_probs": cluster_probs,
        "result_df": result_df,
        "cluster_counts": cluster_counts,
    }

In [ ]:
gmm_6 = perform_gmm_clustering(
    features_pca=pca_results["features_pca"],
    coords=pca_results["coords_clean"],
    n_clusters=6,
    output_dir=output_dir,
)
gmm_6["result_df"].to_csv(os.path.join(output_dir, "gmm_6_clusters.csv"), index=False)

## 函数：cluster 中心混合扩充样本（于 PCA 空间）


In [ ]:
def augment_samples_by_pca_mixing(
    well_data,
    pca_model,
    scaler,
    cluster_results,
    attribute_columns,
    target_column="Thickness of LITHOLOGIES(1: sand)",
    min_samples_per_cluster=3,
    augmentation_factor=2.0,
    min_target_per_cluster=5,
    random_state=42,
    verbose=True,
):
    """
    在PCA空间中使用聚类均值混合法生成伪样本，并反投影到原始特征空间

    参数:
        well_data (DataFrame): 井点数据，包含地震属性和目标变量
        pca_model: 训练好的PCA模型
        scaler: 训练好的标准化模型
        cluster_results (dict): GMM聚类结果，包含cluster_labels和cluster_probs
        attribute_columns (list): 用于聚类和生成的属性列名
        target_column (str): 回归目标列名，默认为"Thickness of LITHOLOGIES(1: sand)"
        min_samples_per_cluster (int): 一个聚类内的最小样本数，默认为3
        augmentation_factor (float): 样本扩增倍数，默认为2.0
        min_target_per_cluster (int): 每个聚类的最小目标样本数，默认为5
        random_state (int): 随机种子，默认为42
        verbose (bool): 是否打印详细信息，默认为True

    返回:
        DataFrame: 扩增后的数据集，包含原始样本和伪样本
    """
    np.random.seed(random_state)

    # 复制原始数据
    well_data = well_data.copy()

    if verbose:
        print(f"使用 {len(attribute_columns)} 个属性列进行样本扩增")
        print(f"原始样本数: {len(well_data)}")
        print(f"目标扩增倍数: {augmentation_factor}倍")

    # 提取属性和目标
    X_orig = well_data[attribute_columns].values
    y_orig = well_data[target_column].values

    # 标准化原始数据
    X_scaled = scaler.transform(X_orig)

    # 降维到PCA空间
    X_pca = pca_model.transform(X_scaled)

    # 获取聚类标签和概率
    cluster_labels = cluster_results["cluster_labels"]

    # 将聚类标签添加到原始数据中
    well_data["Cluster"] = cluster_labels

    # 创建一个空的列表用于存储伪样本
    synthetic_samples = []

    # 统计每个聚类的样本数，并获取唯一的聚类标签
    cluster_counts = well_data["Cluster"].value_counts().to_dict()
    unique_clusters = sorted(cluster_counts.keys())  # 排序后的唯一聚类标签列表

    if verbose:
        print("\n各聚类样本数量:")
        for cluster in unique_clusters:
            print(f"聚类 {cluster}: {cluster_counts[cluster]} 样本")

    # 对每个存在的聚类进行处理
    for cluster_id in unique_clusters:
        # 获取当前聚类的样本
        cluster_mask = cluster_labels == cluster_id
        cluster_data = well_data[cluster_mask]

        # 如果聚类样本数为0，跳过（这应该不会发生，但保留为安全检查）
        if len(cluster_data) == 0:
            if verbose:
                print(f"聚类 {cluster_id} 没有样本，跳过")
            continue

        # 获取PCA空间中的数据点
        X_pca_cluster = X_pca[cluster_mask]

        # 计算当前聚类在PCA空间中的均值
        pca_cluster_mean = np.mean(X_pca_cluster, axis=0)

        # 计算目标值均值和标准差，用于生成新的目标值
        target_mean = cluster_data[target_column].mean()
        target_std = cluster_data[target_column].std()

        # 如果标准差为0或NaN（只有一个样本或所有样本相同），使用一个小的默认值
        if np.isnan(target_std) or target_std < 1e-6:
            target_std = 0.1 * target_mean if target_mean > 0 else 0.1

        # 基于扩增倍数计算目标样本数
        target_samples = max(
            min_target_per_cluster,  # 最小目标样本数
            int(len(cluster_data) * augmentation_factor),  # 基于倍数的目标样本数
        )

        # 计算需要生成的样本数
        n_samples = max(0, target_samples - len(cluster_data))

        if verbose:
            print(
                f"\n聚类 {cluster_id}: 原始样本数 = {len(cluster_data)}, 目标样本数 = {target_samples}, 需要生成 {n_samples} 个伪样本"
            )

        # 如果聚类样本数少于最小阈值，采用特殊处理
        if len(cluster_data) < min_samples_per_cluster:
            if verbose:
                print(f"  - 聚类 {cluster_id} 样本数过少，使用高权重的均值进行混合")

            # 对于样本过少的聚类，生成样本时更偏向均值
            mean_weight_range = (0.6, 0.9)  # 均值的权重范围，偏向更高权重
        else:
            # 对于样本充足的聚类，使用较为均衡的混合比例
            mean_weight_range = (0.3, 0.7)  # 均值的权重范围

        # 生成伪样本
        for i in range(n_samples):
            # 随机选择一个原始样本索引
            orig_idx = np.random.choice(np.where(cluster_mask)[0])
            orig_sample = well_data.iloc[orig_idx]

            # 获取原始样本在PCA空间中的表示
            pca_orig_sample = X_pca[orig_idx]

            # 确定均值的混合权重
            mean_weight = np.random.uniform(mean_weight_range[0], mean_weight_range[1])

            # 在PCA空间中混合生成新样本
            pca_new_sample = mean_weight * pca_cluster_mean + (1 - mean_weight) * pca_orig_sample

            # 反投影回原始空间
            new_sample_scaled = pca_model.inverse_transform(pca_new_sample.reshape(1, -1))
            new_sample_orig = scaler.inverse_transform(new_sample_scaled).flatten()

            # 生成一个新的目标值，略微偏离原始样本的目标值
            # 使用有约束的正态分布，确保目标值在合理范围内
            orig_target = orig_sample[target_column]
            new_target_delta = np.random.normal(0, 0.2 * target_std)

            # 确保混合的目标值与原样本和聚类均值之间具有相似的关系
            target_weight = mean_weight  # 可以使用与特征相同的权重，或单独设置
            new_target_value = target_weight * target_mean + (1 - target_weight) * orig_target + new_target_delta

            # 确保目标值非负（如果是砂厚等物理量）
            new_target_value = max(0, new_target_value)

            # 创建新样本字典
            new_sample_dict = orig_sample.to_dict()
            for j, col in enumerate(attribute_columns):
                new_sample_dict[col] = new_sample_orig[j]
            new_sample_dict[target_column] = new_target_value
            new_sample_dict["Is_Synthetic"] = 1
            new_sample_dict["Cluster"] = cluster_id  # 确保聚类标签也被添加到扩充数据中

            synthetic_samples.append(new_sample_dict)

    # 创建伪样本的DataFrame
    if synthetic_samples:
        synthetic_df = pd.DataFrame(synthetic_samples)

        # 确保原始数据中有Is_Synthetic列
        well_data["Is_Synthetic"] = 0  # 标记为原始样本

        # 合并原始数据和伪样本
        augmented_data = pd.concat([well_data, synthetic_df], ignore_index=True)

        if verbose:
            print(f"\n扩增后的样本数: {len(augmented_data)} (原始: {len(well_data)}, 合成: {len(synthetic_df)})")
            print(f"扩增比例: {len(augmented_data) / len(well_data):.2f}倍")
    else:
        well_data["Is_Synthetic"] = 0
        augmented_data = well_data
        if verbose:
            print("\n没有生成伪样本")

    return augmented_data


def encode_cluster_features(data, cluster_column="Cluster", drop_original=True, prefix="Cluster_"):
    """
    对数据中的聚类标签进行One-Hot编码，并将编码后的特征添加到原始数据中

    参数:
        data (DataFrame): 包含聚类标签的数据框
        cluster_column (str): 聚类标签所在的列名，默认为"Cluster"
        drop_original (bool): 是否删除原始聚类列，默认为True
        prefix (str): One-Hot编码列的前缀，默认为"Cluster_"

    返回:
        DataFrame: 添加了One-Hot编码特征的数据框
    """
    if cluster_column not in data.columns:
        raise ValueError(f"列 '{cluster_column}' 不存在于数据中")

    # 复制原始数据，避免修改原始数据
    result_df = data.copy()

    # 获取唯一的聚类标签
    unique_clusters = sorted(result_df[cluster_column].unique())

    print(f"对'{cluster_column}'列进行One-Hot编码，发现{len(unique_clusters)}个唯一的聚类标签")

    # 使用pandas的get_dummies函数进行One-Hot编码
    cluster_dummies = pd.get_dummies(result_df[cluster_column], prefix=prefix)

    # 合并One-Hot编码结果到原始数据
    result_df = pd.concat([result_df, cluster_dummies], axis=1)

    # 如果需要，删除原始聚类列
    if drop_original:
        result_df = result_df.drop(columns=[cluster_column])
        print(f"已删除原始'{cluster_column}'列")

    print(f"One-Hot编码后数据形状: {result_df.shape}，新增了{len(unique_clusters)}个特征列")

    # 展示部分编码后的特征名
    encoded_cols = [col for col in result_df.columns if col.startswith(prefix)]
    print(f"编码后的特征列: {', '.join(encoded_cols)}")

    return result_df

In [ ]:
# 对井点数据进行PCA降维和GMM聚类
# 注意：这里使用pca_results中的PCA模型和标准化器，但我们需要重新应用到井点数据上

# 确定属性列 - 必须与PCA中使用的列完全一致
attribute_columns = pca_results["features_clean"].columns.tolist()

# 不要只提取特征列，而是保留所有需要的列
X_well = data_H6_2_well_selected.copy()

# 然后对特征列进行预处理
for col in attribute_columns:
    if pd.isna(X_well[col].mean()):
        print(f"列 '{col}' 的均值为NaN，使用0填充")
        X_well[col] = X_well[col].fillna(0)
    else:
        # 使用列均值填充
        X_well[col] = X_well[col].fillna(X_well[col].mean())

# 确保特征列没有剩余的NaN值
if X_well[attribute_columns].isna().any().any():
    print("警告：特征列中仍然存在NaN值，将它们替换为0")
    for col in attribute_columns:
        X_well[col] = X_well[col].fillna(0)

# 对井点数据进行标准化 - 只标准化特征列
X_well_scaled = pca_results["scaler"].transform(X_well[attribute_columns])

# 将井点数据投影到PCA空间
X_well_pca = pca_results["pca"].transform(X_well_scaled)

# 使用已有的GMM模型对井点数据进行聚类
cluster_labels_well = gmm_6["gmm"].predict(X_well_pca)
cluster_probs_well = gmm_6["gmm"].predict_proba(X_well_pca)

# 创建井点数据的聚类结果字典
well_cluster_results = {
    "cluster_labels": cluster_labels_well,
    "cluster_probs": cluster_probs_well,
}

# 使用PCA空间中的中心混合法生成伪样本
augmented_data = augment_samples_by_pca_mixing(
    well_data=X_well,
    pca_model=pca_results["pca"],
    scaler=pca_results["scaler"],
    cluster_results=well_cluster_results,
    attribute_columns=attribute_columns,
    target_column="Thickness of LITHOLOGIES(1: sand)",
    min_samples_per_cluster=2,  # 考虑井数据更少，设置更小的阈值
    augmentation_factor=2.0,  # 目标是原始样本数的2倍
    min_target_per_cluster=5,  # 每个聚类至少5个样本
    random_state=42,
    verbose=True,
)

# 保存扩增后的数据
augmented_data.to_csv(os.path.join(output_dir, "augmented_well_data_pca.csv"), index=False)

# 可视化原始样本和合成样本
plt.figure(figsize=(12, 10))
scatter = plt.scatter(
    augmented_data["X"],
    augmented_data["Y"],
    c=augmented_data["Cluster"],
    s=50 * (1 + augmented_data["Is_Synthetic"]),  # 合成样本点更大
    alpha=0.7,
    cmap="viridis",
    linewidths=0.5 * augmented_data["Is_Synthetic"],  # 合成样本有轮廓
)
plt.colorbar(scatter, label="聚类")
plt.title("原始样本和PCA空间中合成的样本空间分布")
plt.xlabel("X坐标")
plt.ylabel("Y坐标")
plt.savefig(
    os.path.join(output_dir, "pca_augmented_samples_spatial.png"),
    dpi=300,
    bbox_inches="tight",
)
plt.show()

# 可视化目标值分布
plt.figure(figsize=(12, 6))
sns.histplot(
    data=augmented_data,
    x="Thickness of LITHOLOGIES(1: sand)",
    hue="Is_Synthetic",
    element="step",
    bins=15,
    kde=True,
)
plt.title("原始样本和PCA空间中合成样本的目标值分布")
plt.xlabel("砂厚")
plt.savefig(
    os.path.join(output_dir, "pca_augmented_samples_target_dist.png"),
    dpi=300,
    bbox_inches="tight",
)
plt.show()

## 函数：属性间相关性分析


In [ ]:
def analyze_attribute_correlations(
    features_df,
    method="pearson",
    corr_threshold=0.85,
    output_dir="output",
    figsize=(14, 12),
):
    """
    分析属性间的相关性，将高度相关的属性分组，并可视化相关性矩阵

    参数:
        features_df (DataFrame): 包含属性列的数据框
        method (str): 相关系数计算方法，'pearson'或'spearman'，默认为'pearson'
        corr_threshold (float): 相关性阈值，高于此值的属性被视为高度相关，默认为0.85
        output_dir (str): 输出图像的目录，默认为"output"
        figsize (tuple): 图像尺寸，默认为(14, 12)

    返回:
        list: 相关属性组列表，每组包含高度相关的属性
    """
    print(f"======== 属性相关性分析开始 (方法: {method}) ========")
    print(f"属性数量: {features_df.shape[1]}")

    # 计算相关系数矩阵
    corr_matrix = features_df.corr(method=method)

    # 可视化相关系数矩阵
    plt.figure(figsize=figsize)
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # 创建上三角掩码
    cmap = sns.diverging_palette(230, 20, as_cmap=True)  # 使用发散色调色板

    sns.heatmap(
        corr_matrix,
        mask=mask,
        cmap=cmap,
        vmax=1.0,
        vmin=-1.0,
        center=0,
        square=True,
        linewidths=0.5,
        annot=False,  # 不显示具体数值，避免过于拥挤
        cbar_kws={"shrink": 0.8},
    )

    plt.title(f"属性相关性矩阵 ({method}相关系数)", fontsize=16)
    plt.tight_layout()
    plt.savefig(
        os.path.join(output_dir, f"attribute_correlation_{method}.png"),
        dpi=300,
        bbox_inches="tight",
    )
    plt.show()

    # 提取相关属性组
    # 为避免重复，我们只考虑上三角矩阵(不包括对角线)
    n_attrs = len(corr_matrix.columns)
    correlated_groups = []
    processed_attrs = set()  # 用于跟踪已经分配到组的属性

    # 对每个属性，寻找与其高度相关的其他属性
    for i, attr1 in enumerate(corr_matrix.columns):
        if attr1 in processed_attrs:
            continue

        # 初始化一个新组，包含当前属性
        correlated_group = {attr1}
        processed_attrs.add(attr1)

        # 查找与当前属性高度相关的所有其他属性
        for j, attr2 in enumerate(corr_matrix.columns):
            if i != j and attr2 not in processed_attrs:
                corr_value = abs(corr_matrix.iloc[i, j])
                if corr_value >= corr_threshold:
                    correlated_group.add(attr2)
                    processed_attrs.add(attr2)

        # 如果组中有多个属性（即找到了相关属性），则将其添加到结果列表
        if len(correlated_group) > 1:
            correlated_groups.append(correlated_group)

    # 添加未分组的单个属性
    for attr in corr_matrix.columns:
        if attr not in processed_attrs:
            correlated_groups.append({attr})
            processed_attrs.add(attr)

    # 输出相关属性组
    print("\n高度相关的属性组 (相关系数阈值 >= {}):".format(corr_threshold))
    for i, group in enumerate(correlated_groups):
        if len(group) > 1:
            print(f"\n组 {i + 1} (包含 {len(group)} 个属性):")
            for attr in group:
                print(f"  - {attr}")

    # 输出独立属性（未分组）
    independent_attrs = [list(group)[0] for group in correlated_groups if len(group) == 1]
    print(f"\n独立属性 ({len(independent_attrs)} 个):")
    for i, attr in enumerate(independent_attrs):
        print(f"  - {attr}")

    # 计算每组内部的平均相关系数，用于评估组内相关程度
    if len(correlated_groups) > 0 and any(len(group) > 1 for group in correlated_groups):
        print("\n各组内部平均相关系数:")
        for i, group in enumerate(correlated_groups):
            if len(group) > 1:
                group_list = list(group)
                group_corr = corr_matrix.loc[group_list, group_list].values
                # 计算组内非对角线元素的平均绝对值
                mean_corr = np.sum(np.abs(group_corr)) / (len(group_list) ** 2 - len(group_list))
                print(f"  - 组 {i + 1}: {mean_corr:.4f}")

    print(f"======== 属性相关性分析完成 ========")

    return correlated_groups

In [ ]:
# 分析原始特征的相关性
correlated_attribute_groups = analyze_attribute_correlations(
    features_df=pca_results["features_clean"],  # 使用PCA结果中的清理后特征
    method="pearson",  # 使用Pearson相关系数
    corr_threshold=0.85,  # 相关性阈值设为0.85
    output_dir=output_dir,  # 输出目录
    figsize=(16, 14),  # 图像尺寸
)

# 输出每个组的特征数量统计
group_sizes = [len(group) for group in correlated_attribute_groups]
print(f"\n相关属性组统计:")
print(f"  - 总组数: {len(correlated_attribute_groups)}")
print(f"  - 最大组大小: {max(group_sizes)} 属性")
print(f"  - 包含多个属性的组数: {sum(1 for size in group_sizes if size > 1)}")
print(f"  - 平均组大小: {sum(group_sizes) / len(group_sizes):.2f} 属性")

## 按组进行 RF importance 分析


In [ ]:
def analyze_rf_importance_by_group(
    well_data,
    attribute_groups,
    target_column="Thickness of LITHOLOGIES(1: sand)",
    top_n=1,
    test_size=0.3,
    n_estimators=100,
    random_state=42,
    output_dir="output",
    verbose=True,
):
    """
    对每组属性进行随机森林重要性分析，选择每组中最重要的特征

    参数:
        well_data (DataFrame): 井点数据，包含地震属性和目标变量
        attribute_groups (list): 属性组列表，每组是一个包含相关属性的集合
        target_column (str): 目标变量列名，默认为"Thickness of LITHOLOGIES(1: sand)"
        top_n (int): 从每组中选择的最重要特征数量，默认为1
        test_size (float): 测试集比例，默认为0.3
        n_estimators (int): 随机森林中的树数量，默认为100
        random_state (int): 随机种子，默认为42
        output_dir (str): 输出目录，默认为"output"
        verbose (bool): 是否输出详细信息，默认为True

    返回:
        list: 所有组中选出的最重要特征列表
    """
    print("======== 按组进行随机森林特征重要性分析 ========")

    # 确保目标列存在
    if target_column not in well_data.columns:
        print(f"错误: 目标列 '{target_column}' 不存在于数据中")
        return []

    # 准备目标变量
    y = well_data[target_column].values

    # 初始化结果存储
    selected_features = []
    all_importances = {}

    # 对每组属性进行分析
    for i, group in enumerate(attribute_groups):
        group_list = list(group)

        # 如果组中只有一个属性，直接添加到结果中
        if len(group_list) == 1:
            selected_features.append(group_list[0])
            if verbose:
                print(f"\n组 {i + 1} 只有一个属性: {group_list[0]}，直接添加到选择列表")
            continue

        # 打印当前组信息
        if verbose:
            print(f"\n分析组 {i + 1} (包含 {len(group_list)} 个属性):")
            for attr in group_list:
                print(f"  - {attr}")

        # 提取当前组的特征
        X_group = well_data[group_list].values

        # 划分训练集和测试集
        X_train, X_test, y_train, y_test = train_test_split(X_group, y, test_size=test_size, random_state=random_state)

        # 训练随机森林模型
        rf = RandomForestRegressor(
            n_estimators=n_estimators,
            random_state=random_state,
            n_jobs=-1,  # 使用所有CPU核心
        )
        rf.fit(X_train, y_train)

        # 评估模型性能
        y_pred = rf.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        if verbose:
            print(f"  组 {i + 1} 模型性能: MSE = {mse:.4f}, R² = {r2:.4f}")

        # 获取特征重要性
        importances = rf.feature_importances_
        importance_dict = dict(zip(group_list, importances))

        # 按重要性排序
        sorted_importances = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

        # 保存每个属性的重要性
        for attr, imp in sorted_importances:
            all_importances[attr] = imp

        # 输出该组属性的重要性
        if verbose:
            print(f"  组 {i + 1} 特征重要性:")
            for attr, imp in sorted_importances:
                print(f"    - {attr}: {imp:.6f}")

        # 选择 top_n 个重要性最高的特征
        top_features = [attr for attr, _ in sorted_importances[: min(top_n, len(sorted_importances))]]
        selected_features.extend(top_features)

        if verbose:
            print(f"  从组 {i + 1} 中选择的特征: {', '.join(top_features)}")

        # 可视化特征重要性
        plt.figure(figsize=(10, max(6, len(group_list) * 0.5)))
        y_pos = np.arange(len(group_list))

        # 排序后的重要性和属性名
        sorted_attrs = [attr for attr, _ in sorted_importances]
        sorted_imps = [imp for _, imp in sorted_importances]

        # 绘制条形图
        bars = plt.barh(y_pos, sorted_imps, align="center")

        # 为top_n特征着色
        for j in range(min(top_n, len(bars))):
            bars[j].set_color("red")

        plt.yticks(y_pos, sorted_attrs)
        plt.xlabel("特征重要性")
        plt.title(f"组 {i + 1} 的特征重要性")
        plt.tight_layout()
        plt.savefig(
            os.path.join(output_dir, f"group_{i + 1}_feature_importance.png"),
            dpi=300,
            bbox_inches="tight",
        )
        plt.show()

    # 可视化所有选择的特征及其重要性
    selected_importances = {feat: all_importances.get(feat, 0) for feat in selected_features}
    sorted_selected = sorted(selected_importances.items(), key=lambda x: x[1], reverse=True)

    plt.figure(figsize=(12, max(6, len(selected_features) * 0.4)))
    y_pos = np.arange(len(selected_features))

    # 排序后的重要性和属性名
    sorted_selected_attrs = [attr for attr, _ in sorted_selected]
    sorted_selected_imps = [imp for _, imp in sorted_selected]

    # 绘制条形图
    plt.barh(y_pos, sorted_selected_imps, align="center")
    plt.yticks(y_pos, sorted_selected_attrs)
    plt.xlabel("特征重要性")
    plt.title("所有选择特征的重要性")
    plt.tight_layout()
    plt.savefig(
        os.path.join(output_dir, "selected_features_importance.png"),
        dpi=300,
        bbox_inches="tight",
    )
    plt.show()

    # 输出最终选择的特征
    print("\n最终选择的特征列表:")
    for i, feature in enumerate(sorted_selected_attrs):
        print(f"{i + 1}. {feature}: {selected_importances[feature]:.6f}")

    print(f"\n共选择了 {len(selected_features)} 个特征用于建模")
    print("======== 随机森林特征重要性分析完成 ========")

    return sorted_selected_attrs

In [ ]:
# 在井点数据上应用随机森林重要性分析
# 使用原始井点数据，不使用扩增数据
selected_features = analyze_rf_importance_by_group(
    well_data=data_H6_2_well_selected,  # 原始井点数据
    attribute_groups=correlated_attribute_groups,  # 前面生成的属性组
    target_column="Thickness of LITHOLOGIES(1: sand)",
    top_n=1,  # 每组选择1个最重要的特征
    test_size=0.3,
    n_estimators=100,
    random_state=42,
    output_dir=output_dir,
    verbose=True,
)

## 函数：SVR


In [ ]:
def build_svr_regression_model(
    data,
    selected_features,
    target_column="Thickness of LITHOLOGIES(1: sand)",
    include_cluster_probs=True,
    test_size=0.3,
    random_state=42,
    output_dir="output",
    threshold_zero=0.1,  # 小于此值视为0
    n_splits=5,
    verbose=True,
):
    """
    使用选定特征和SVR构建回归模型

    参数:
        data (DataFrame): 包含特征和目标变量的数据框
        selected_features (list): 选定的特征列表
        target_column (str): 目标变量列名
        include_cluster_probs (bool): 是否包含聚类概率作为附加特征
        test_size (float): 测试集比例
        random_state (int): 随机种子
        output_dir (str): 输出目录
        threshold_zero (float): 砂厚小于此阈值视为0
        n_splits (int): 交叉验证折数
        verbose (bool): 是否输出详细信息

    返回:
        dict: 包含模型、预测结果和评估指标的字典
    """
    print("======== SVR回归建模开始 ========")

    # 复制数据，避免修改原始数据
    data_copy = data.copy()

    # 准备特征和目标
    X = data_copy[selected_features].values
    y = data_copy[target_column].values

    # 创建一个布尔变量，表示目标是否为0（或近似为0）
    y_is_zero = y <= threshold_zero

    # 如果需要，添加聚类概率作为附加特征
    if include_cluster_probs and "cluster_probs" in data_copy.columns:
        if verbose:
            print("添加聚类概率作为附加特征")

        # 假设cluster_probs是一个存储概率向量的列
        # 我们需要将其转换为单独的列
        cluster_probs = np.array(data_copy["cluster_probs"].tolist())
        X = np.hstack((X, cluster_probs))

    # 使用分层采样进行训练/测试集划分
    # 我们使用y_is_zero作为分层依据，确保"砂厚=0"和"砂厚>0"样本的比例一致
    X_train, X_test, y_train, y_test, y_train_zero, y_test_zero = train_test_split(
        X,
        y,
        y_is_zero,
        test_size=test_size,
        random_state=random_state,
        stratify=y_is_zero,  # 使用是否为0作为分层依据
    )

    if verbose:
        print(f"训练集大小: {X_train.shape[0]} 样本")
        print(f"测试集大小: {X_test.shape[0]} 样本")
        print(f"训练集中砂厚为0的样本比例: {np.mean(y_train_zero):.2%}")
        print(f"测试集中砂厚为0的样本比例: {np.mean(y_test_zero):.2%}")

    # 标准化特征
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 使用GridSearchCV优化SVR参数
    # 定义参数网格
    param_grid = {
        "C": [0.1, 1, 10, 100],
        "epsilon": [0.01, 0.1, 0.2, 0.5],
        "kernel": ["linear", "rbf", "poly"],
        "gamma": ["scale", "auto", 0.1, 0.01],
    }

    # 创建SVR模型
    svr = SVR()

    # 创建分层K折交叉验证，保持每折中"砂厚=0"和"砂厚>0"样本的比例一致
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # 创建一个GroupKFold用于保持砂厚为0和非0的分组
    # 首先创建分组标签
    groups = np.zeros(len(y_train))
    groups[y_train > threshold_zero] = 1  # 非零样本分为一组

    grid_search = GridSearchCV(
        svr,
        param_grid,
        cv=kfold,  # 使用普通KFold
        scoring="neg_mean_squared_error",
        n_jobs=-1,
        verbose=2 if verbose else 0,
    )

    # 训练模型
    print("\n正在训练SVR模型并优化参数...")
    grid_search.fit(X_train_scaled, y_train)

    # 获取最佳模型
    best_svr = grid_search.best_estimator_
    best_params = grid_search.best_params_

    if verbose:
        print("\n最佳SVR参数:")
        for param, value in best_params.items():
            print(f"  - {param}: {value}")

    # 在测试集上进行预测
    y_pred = best_svr.predict(X_test_scaled)

    # 计算评估指标
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print("\n模型评估指标:")
    print(f"  - R²: {r2:.4f}")
    print(f"  - MAE: {mae:.4f}")
    print(f"  - RMSE: {rmse:.4f}")

    # 计算在"真实值=0"的样本中预测是否也为0的准确率
    y_pred_is_zero = y_pred <= threshold_zero
    zero_accuracy = accuracy_score(y_test_zero, y_pred_is_zero)

    # 计算更详细的指标
    # 真实为0，预测为0的比例 (真阴性率)
    true_negative_rate = np.sum((y_test <= threshold_zero) & (y_pred <= threshold_zero)) / np.sum(
        y_test <= threshold_zero
    )
    # 真实不为0，预测不为0的比例 (真阳性率)
    true_positive_rate = np.sum((y_test > threshold_zero) & (y_pred > threshold_zero)) / np.sum(y_test > threshold_zero)

    print(f"\n砂厚判别能力 (阈值 = {threshold_zero}):")
    print(f"  - 总体准确率: {zero_accuracy:.4f}")
    print(f"  - 真实为0时预测为0的比例: {true_negative_rate:.4f}")
    print(f"  - 真实不为0时预测不为0的比例: {true_positive_rate:.4f}")

    # 创建真值vs预测值散点图
    plt.figure(figsize=(10, 8))

    # 绘制散点图，区分"砂厚=0"和"砂厚>0"的样本
    plt.scatter(
        y_test[~y_test_zero],
        y_pred[~y_test_zero],
        c="blue",
        alpha=0.6,
        label="砂厚 > 0",
    )
    plt.scatter(y_test[y_test_zero], y_pred[y_test_zero], c="red", alpha=0.6, label="砂厚 ≈ 0")

    # 绘制对角线 (y=x)
    max_val = max(np.max(y_test), np.max(y_pred))
    plt.plot([0, max_val], [0, max_val], "k--", label="理想预测")

    # 绘制阈值线
    plt.axhline(y=threshold_zero, color="green", linestyle=":", label=f"阈值 = {threshold_zero}")
    plt.axvline(x=threshold_zero, color="green", linestyle=":")

    plt.xlabel("真实砂厚")
    plt.ylabel("预测砂厚")
    plt.title("SVR回归：真实值 vs 预测值")
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 保存图像
    plt.savefig(os.path.join(output_dir, "svr_true_vs_pred.png"), dpi=300, bbox_inches="tight")
    plt.show()

    # 创建误差直方图
    errors = y_pred - y_test
    plt.figure(figsize=(10, 6))
    plt.hist(errors, bins=30, alpha=0.7, color="blue")
    plt.axvline(x=0, color="red", linestyle="--")
    plt.xlabel("预测误差 (预测值 - 真实值)")
    plt.ylabel("频数")
    plt.title("SVR回归预测误差分布")
    plt.grid(True, alpha=0.3)
    plt.savefig(
        os.path.join(output_dir, "svr_error_distribution.png"),
        dpi=300,
        bbox_inches="tight",
    )
    plt.show()

    # 将结果保存到CSV文件
    results_df = pd.DataFrame(
        {
            "True_Value": y_test,
            "Predicted_Value": y_pred,
            "Is_Zero_True": y_test_zero,
            "Is_Zero_Pred": y_pred_is_zero,
            "Error": errors,
        }
    )
    results_df.to_csv(os.path.join(output_dir, "svr_prediction_results.csv"), index=False)

    print("======== SVR回归建模完成 ========")

    # 返回结果字典
    return {
        "model": best_svr,
        "scaler": scaler,
        "best_params": best_params,
        "metrics": {
            "r2": r2,
            "mae": mae,
            "rmse": rmse,
            "zero_accuracy": zero_accuracy,
            "true_negative_rate": true_negative_rate,
            "true_positive_rate": true_positive_rate,
        },
        "predictions": {
            "y_test": y_test,
            "y_pred": y_pred,
            "y_test_zero": y_test_zero,
            "y_pred_zero": y_pred_is_zero,
        },
    }

In [ ]:
# 使用选择的特征构建SVR回归模型
svr_results = build_svr_regression_model(
    data=augmented_data,  # 可以使用扩增后的数据
    selected_features=selected_features,  # 使用之前按组选择的特征
    target_column="Thickness of LITHOLOGIES(1: sand)",
    include_cluster_probs=False,  # 是否包含聚类概率（如果有）
    test_size=0.3,
    random_state=42,
    output_dir=output_dir,
    threshold_zero=0.1,  # 砂厚小于0.1视为0
    n_splits=5,
    verbose=True,
)

# 使用原始井点数据构建SVR回归模型
# svr_results = build_svr_regression_model(
#     data=data_H6_2_well_selected,  # 使用原始井点数据
#     selected_features=selected_features,
#     target_column="Thickness of LITHOLOGIES(1: sand)",
#     include_cluster_probs=False,
#     test_size=0.3,
#     random_state=42,
#     output_dir=output_dir,
#     threshold_zero=0.1,
#     n_splits=5,
#     verbose=True,
# )

# 可视化最重要的参数影响
if svr_results["best_params"]["kernel"] == "rbf":
    print("\n分析C和gamma参数对模型性能的影响...")

    # 创建不同C和gamma值的参数网格
    C_range = np.logspace(-1, 2, 4)
    gamma_range = ["scale", "auto"] + list(np.logspace(-2, 0, 3))

    # 为了简化，使用最佳epsilon
    epsilon = svr_results["best_params"]["epsilon"]

    # 创建结果表格
    param_scores = []

    # 重新提取特征和目标变量
    X = data_H6_2_well_selected[selected_features].values
    y = data_H6_2_well_selected["Thickness of LITHOLOGIES(1: sand)"].values

    # 再次划分训练集和测试集（使用相同的随机种子确保一致性）
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # 标准化特征
    scaler = svr_results["scaler"]  # 使用已有的标准化器
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    for C in C_range:
        for gamma in gamma_range:
            if isinstance(gamma, str):
                gamma_label = gamma
            else:
                gamma_label = f"{gamma:.3f}"

            svr = SVR(kernel="rbf", C=C, epsilon=epsilon, gamma=gamma)
            svr.fit(X_train_scaled, y_train)
            score = svr.score(X_test_scaled, y_test)
            param_scores.append({"C": C, "gamma": gamma_label, "R²": score})

    # 转换为DataFrame并打印
    param_df = pd.DataFrame(param_scores)
    param_df = param_df.pivot(index="C", columns="gamma", values="R²")
    print("\nC和gamma参数对R²的影响:")
    print(param_df)

    # 将参数影响保存到CSV
    param_df.to_csv(os.path.join(output_dir, "svr_parameter_influence.csv"))

## 预测


In [ ]:
# 对整个地震数据区域进行预测
print("======== 开始预测整个地震区域的砂厚 ========")

# 准备地震属性数据进行预测
# 首先对属性进行预处理（同样的方式，确保一致性）
seismic_features = preprocess_features(
    data=data_H6_2_attr, attribute_columns=selected_features, missing_values=[-999], verbose=True
)

# 提取坐标信息用于最终结果输出
seismic_coords = data_H6_2_attr[["X", "Y", "Z"]].copy()

# 只保留选定的特征
seismic_features_selected = seismic_features[selected_features]

# 将特征转换为numpy数组
X_seismic = seismic_features_selected.values

# 使用训练好的标准化器对特征进行标准化
X_seismic_scaled = svr_results["scaler"].transform(X_seismic)

# 使用训练好的SVR模型进行预测
print(f"使用SVR模型预测{len(X_seismic)}个点的砂厚...")
sand_thickness_pred = svr_results["model"].predict(X_seismic_scaled)

# 创建预测结果DataFrame
prediction_results = pd.DataFrame(
    {
        "X": seismic_coords["X"],
        "Y": seismic_coords["Y"],
        "Z": seismic_coords["Z"],
        "Predicted_Sand_Thickness": sand_thickness_pred,
    }
)

# 为负值预测结果置为0（如果有）
if (prediction_results["Predicted_Sand_Thickness"] < 0).any():
    neg_count = (prediction_results["Predicted_Sand_Thickness"] < 0).sum()
    print(f"有{neg_count}个点的预测砂厚为负值，已将其置为0")
    prediction_results["Predicted_Sand_Thickness"] = prediction_results["Predicted_Sand_Thickness"].clip(lower=0)

# 统计预测结果
pred_zero_count = (prediction_results["Predicted_Sand_Thickness"] <= 0.1).sum()
pred_nonzero_count = (prediction_results["Predicted_Sand_Thickness"] > 0.1).sum()
pred_mean = prediction_results["Predicted_Sand_Thickness"].mean()
pred_max = prediction_results["Predicted_Sand_Thickness"].max()

print("\n预测结果统计:")
print(f"  - 总点数: {len(prediction_results)}")
print(f"  - 预测砂厚≤0.1的点数: {pred_zero_count} ({pred_zero_count / len(prediction_results):.2%})")
print(f"  - 预测砂厚>0.1的点数: {pred_nonzero_count} ({pred_nonzero_count / len(prediction_results):.2%})")
print(f"  - 平均预测砂厚: {pred_mean:.4f}")
print(f"  - 最大预测砂厚: {pred_max:.4f}")

# 保存预测结果到CSV文件
prediction_file = os.path.join(output_dir, "H6-2_predicted_sand_thickness.csv")
prediction_results.to_csv(prediction_file, index=False)
print(f"\n预测结果已保存到文件: {prediction_file}")

# 可视化预测结果 - 散点图
plt.figure(figsize=(14, 12))
scatter = plt.scatter(
    prediction_results["X"],
    prediction_results["Y"],
    c=prediction_results["Predicted_Sand_Thickness"],
    cmap="viridis",
    s=2,  # 点大小较小，因为点数可能很多
    alpha=0.7,
    vmin=0,
    vmax=min(pred_max, pred_mean * 3),  # 限制颜色范围，便于观察
)
plt.colorbar(scatter, label="预测砂厚")
plt.title("H6-2层砂厚预测结果空间分布")
plt.xlabel("X坐标")
plt.ylabel("Y坐标")
plt.savefig(os.path.join(output_dir, "H6-2_predicted_sand_thickness_map.png"), dpi=300, bbox_inches="tight")
plt.show()

# 可视化预测结果 - 直方图
plt.figure(figsize=(12, 6))
plt.hist(prediction_results["Predicted_Sand_Thickness"], bins=50, alpha=0.7, color="blue")
plt.axvline(x=0.1, color="red", linestyle="--", label="阈值 = 0.1")
plt.xlabel("预测砂厚")
plt.ylabel("频数")
plt.title("H6-2层砂厚预测结果分布")
plt.grid(True, alpha=0.3)
plt.legend()
plt.savefig(os.path.join(output_dir, "H6-2_predicted_sand_thickness_histogram.png"), dpi=300, bbox_inches="tight")
plt.show()

# 导出为Petrel可读的XYZ格式文件
petrel_output_file = os.path.join(output_dir, "H6-2_predicted_sand_thickness_petrel.txt")
with open(petrel_output_file, "w") as f:
    # 写入标题
    f.write("# H6-2层砂厚预测结果\n")
    f.write("# X Y Z Predicted_Sand_Thickness\n")

    # 写入数据
    for _, row in prediction_results.iterrows():
        f.write(f"{row['X']:.6f} {row['Y']:.6f} {row['Z']:.6f} {row['Predicted_Sand_Thickness']:.6f}\n")

print(f"预测结果已保存为Petrel可导入的XYZ格式: {petrel_output_file}")

print("======== 砂厚预测完成 ========")

## 排查问题


In [ ]:
def train_and_predict_svr(
    train_data,
    predict_data,
    selected_features,
    target_column="Thickness of LITHOLOGIES(1: sand)",
    test_size=0.3,
    random_state=42,
    output_dir="output",
    threshold_zero=0.1,
    n_splits=5,
    verbose=True,
):
    """
    SVR模型训练和预测的整合函数，包含详细日志用于问题诊断

    参数:
        train_data (DataFrame): 用于训练的井点数据
        predict_data (DataFrame): 用于预测的地震数据
        selected_features (list): 选定的特征列表
        target_column (str): 目标变量列名
        test_size (float): 测试集比例
        random_state (int): 随机种子
        output_dir (str): 输出目录
        threshold_zero (float): 砂厚小于此阈值视为0
        n_splits (int): 交叉验证折数
        verbose (bool): 是否输出详细信息

    返回:
        dict: 包含训练结果、预测结果和诊断信息的字典
    """
    print("======== SVR回归模型训练与预测开始 ========")
    print(f"训练数据: {train_data.shape}, 预测数据: {predict_data.shape}")
    print(f"选定特征 ({len(selected_features)}): {selected_features}")

    # 提取训练特征和目标
    X_train_raw = train_data[selected_features].values
    y_train_all = train_data[target_column].values

    # 诊断日志：训练数据基本统计信息
    print("\n训练数据统计信息:")
    for i, feature in enumerate(selected_features):
        feature_values = X_train_raw[:, i]
        print(
            f"  - {feature}: 均值={np.mean(feature_values):.4f}, 标准差={np.std(feature_values):.4f}, "
            f"最小值={np.min(feature_values):.4f}, 最大值={np.max(feature_values):.4f}"
        )

    print(
        f"  - 目标变量: 均值={np.mean(y_train_all):.4f}, 标准差={np.std(y_train_all):.4f}, "
        f"最小值={np.min(y_train_all):.4f}, 最大值={np.max(y_train_all):.4f}"
    )

    # 是否为0（或近似为0）的标记
    y_is_zero = y_train_all <= threshold_zero
    print(f"  - 训练集中砂厚≤{threshold_zero}的样本比例: {np.mean(y_is_zero):.2%}")

    # 预处理预测数据特征
    seismic_features = preprocess_features(
        data=predict_data,
        attribute_columns=selected_features,
        missing_values=[-999],
        verbose=verbose,
    )

    # 只保留选定特征
    X_seismic_raw = seismic_features[selected_features].values

    # 诊断日志：预测数据基本统计信息
    print("\n预测数据统计信息:")
    for i, feature in enumerate(selected_features):
        feature_values = X_seismic_raw[:, i]
        print(
            f"  - {feature}: 均值={np.mean(feature_values):.4f}, 标准差={np.std(feature_values):.4f}, "
            f"最小值={np.min(feature_values):.4f}, 最大值={np.max(feature_values):.4f}"
        )

    # 训练/测试集划分
    X_train, X_test, y_train, y_test, y_train_zero, y_test_zero = train_test_split(
        X_train_raw,
        y_train_all,
        y_is_zero,
        test_size=test_size,
        random_state=random_state,
        stratify=y_is_zero,
    )

    print(f"\n训练集大小: {X_train.shape[0]} 样本, 测试集大小: {X_test.shape[0]} 样本")

    # 标准化特征
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 诊断日志：标准化后的统计信息
    print("\n标准化后的训练数据统计信息:")
    for i, feature in enumerate(selected_features):
        feature_values = X_train_scaled[:, i]
        print(
            f"  - {feature}: 均值={np.mean(feature_values):.4f}, 标准差={np.std(feature_values):.4f}, "
            f"最小值={np.min(feature_values):.4f}, 最大值={np.max(feature_values):.4f}"
        )

    # 标准化预测数据
    X_seismic_scaled = scaler.transform(X_seismic_raw)

    # 诊断日志：标准化后的预测数据
    print("\n标准化后的预测数据统计信息:")
    for i, feature in enumerate(selected_features):
        feature_values = X_seismic_scaled[:, i]
        print(
            f"  - {feature}: 均值={np.mean(feature_values):.4f}, 标准差={np.std(feature_values):.4f}, "
            f"最小值={np.min(feature_values):.4f}, 最大值={np.max(feature_values):.4f}"
        )

    # 定义参数网格
    param_grid = {
        "C": [0.1, 1, 10, 100],
        "epsilon": [0.01, 0.1, 0.2, 0.5],
        "kernel": ["linear", "rbf", "poly"],
        "gamma": ["scale", "auto", 0.1, 0.01],
    }

    # 创建SVR模型
    svr = SVR()
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    grid_search = GridSearchCV(
        svr,
        param_grid,
        cv=kfold,
        scoring="neg_mean_squared_error",
        n_jobs=-1,
        verbose=2 if verbose else 0,
    )

    # 训练模型
    print("\n正在训练SVR模型并优化参数...")
    grid_search.fit(X_train_scaled, y_train)

    # 获取最佳模型
    best_svr = grid_search.best_estimator_
    best_params = grid_search.best_params_

    print("\n最佳SVR参数:")
    for param, value in best_params.items():
        print(f"  - {param}: {value}")

    # 测试集上评估
    y_pred_test = best_svr.predict(X_test_scaled)

    # 计算测试集评估指标
    r2 = r2_score(y_test, y_pred_test)
    mae = mean_absolute_error(y_test, y_pred_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

    print("\n测试集评估指标:")
    print(f"  - R²: {r2:.4f}")
    print(f"  - MAE: {mae:.4f}")
    print(f"  - RMSE: {rmse:.4f}")

    # 使用最佳模型在训练集上预测（检查过拟合）
    y_pred_train = best_svr.predict(X_train_scaled)
    r2_train = r2_score(y_train, y_pred_train)

    print(f"\n训练集 R²: {r2_train:.4f}（用于检查过拟合）")

    # 训练/测试集预测值统计
    print("\n训练集预测值统计:")
    print(f"  - 均值: {np.mean(y_pred_train):.4f}")
    print(f"  - 标准差: {np.std(y_pred_train):.4f}")
    print(f"  - 最小值: {np.min(y_pred_train):.4f}")
    print(f"  - 最大值: {np.max(y_pred_train):.4f}")
    print(f"  - 预测为0的比例: {np.mean(y_pred_train <= threshold_zero):.2%}")

    print("\n测试集预测值统计:")
    print(f"  - 均值: {np.mean(y_pred_test):.4f}")
    print(f"  - 标准差: {np.std(y_pred_test):.4f}")
    print(f"  - 最小值: {np.min(y_pred_test):.4f}")
    print(f"  - 最大值: {np.max(y_pred_test):.4f}")
    print(f"  - 预测为0的比例: {np.mean(y_pred_test <= threshold_zero):.2%}")

    # 调试：检查特征对模型影响
    if hasattr(best_svr, "coef_"):
        # 线性核心有系数可供查看
        print("\n特征系数（重要性）:")
        for feature, coef in zip(selected_features, best_svr.coef_[0]):
            print(f"  - {feature}: {coef:.6f}")

    # 使用前10个样本进行预测，输出详细结果
    print("\n预测数据前10个样本的详细结果:")
    for i in range(min(10, X_seismic_scaled.shape[0])):
        pred_value = best_svr.predict([X_seismic_scaled[i]])[0]
        print(f"样本 {i + 1}: 预测值 = {pred_value:.4f}")
        print("  特征值:")
        for j, feature in enumerate(selected_features):
            print(f"    - {feature}: 原始值={X_seismic_raw[i, j]:.4f}, 标准化值={X_seismic_scaled[i, j]:.4f}")

    # 对整个地震数据进行预测
    print(f"\n使用SVR模型预测{len(X_seismic_raw)}个点的砂厚...")
    sand_thickness_pred = best_svr.predict(X_seismic_scaled)

    # 预测值统计分析
    unique_values = np.unique(sand_thickness_pred)
    print(f"预测结果中的唯一值数量: {len(unique_values)}")
    if len(unique_values) <= 10:
        print(f"所有唯一预测值: {unique_values}")
    else:
        print(f"前10个唯一预测值: {unique_values[:10]}")

    # 检查是否所有预测值都相同
    if len(unique_values) == 1:
        print("\n警告: 所有点的预测值都是相同的!")

        # 尝试使用不同的模型参数预测一小部分数据
        print("\n尝试使用不同参数的SVR模型:")
        test_params = [
            {"kernel": "linear", "C": 1.0, "epsilon": 0.1},
            {"kernel": "rbf", "C": 10.0, "gamma": 0.01, "epsilon": 0.1},
            {"kernel": "poly", "C": 1.0, "degree": 2, "epsilon": 0.1},
        ]

        for params in test_params:
            test_svr = SVR(**params)
            test_svr.fit(X_train_scaled, y_train)
            test_pred = test_svr.predict(X_seismic_scaled[:100])  # 只预测前100个样本
            unique_test = np.unique(test_pred)
            print(
                f"  - SVR参数 {params}: 唯一值数量={len(unique_test)}, "
                f"范围=[{np.min(test_pred):.4f}, {np.max(test_pred):.4f}]"
            )

    # 创建预测结果DataFrame
    prediction_results = pd.DataFrame(
        {
            "X": predict_data["X"],
            "Y": predict_data["Y"],
            "Z": predict_data["Z"],
            "Predicted_Sand_Thickness": sand_thickness_pred,
        }
    )

    # 为负值预测结果置为0（如果有）
    if (prediction_results["Predicted_Sand_Thickness"] < 0).any():
        neg_count = (prediction_results["Predicted_Sand_Thickness"] < 0).sum()
        print(f"有{neg_count}个点的预测砂厚为负值，已将其置为0")
        prediction_results["Predicted_Sand_Thickness"] = prediction_results["Predicted_Sand_Thickness"].clip(lower=0)

    # 统计预测结果
    pred_zero_count = (prediction_results["Predicted_Sand_Thickness"] <= threshold_zero).sum()
    pred_nonzero_count = (prediction_results["Predicted_Sand_Thickness"] > threshold_zero).sum()
    pred_mean = prediction_results["Predicted_Sand_Thickness"].mean()
    pred_max = prediction_results["Predicted_Sand_Thickness"].max()

    print("\n预测结果统计:")
    print(f"  - 总点数: {len(prediction_results)}")
    print(f"  - 预测砂厚≤{threshold_zero}的点数: {pred_zero_count} ({pred_zero_count / len(prediction_results):.2%})")
    print(
        f"  - 预测砂厚>{threshold_zero}的点数: {pred_nonzero_count} ({pred_nonzero_count / len(prediction_results):.2%})"
    )
    print(f"  - 平均预测砂厚: {pred_mean:.4f}")
    print(f"  - 最大预测砂厚: {pred_max:.4f}")

    # 保存预测结果到CSV文件
    prediction_file = os.path.join(output_dir, "H6-2_predicted_sand_thickness.csv")
    prediction_results.to_csv(prediction_file, index=False)
    print(f"\n预测结果已保存到文件: {prediction_file}")

    print("======== SVR回归模型训练与预测完成 ========")

    return {
        "model": best_svr,
        "best_params": best_params,
        "scaler": scaler,
        "metrics": {
            "r2_test": r2,
            "r2_train": r2_train,
            "mae": mae,
            "rmse": rmse,
        },
        "predictions": {
            "train": y_pred_train,
            "test": y_pred_test,
            "seismic": sand_thickness_pred,
        },
        "prediction_results": prediction_results,
    }

In [ ]:
# # 使用整合的函数进行SVR模型训练与预测
# svr_analysis = train_and_predict_svr(
#     train_data=data_H6_2_well_selected,  # 训练数据：井点数据
#     predict_data=data_H6_2_attr,  # 预测数据：地震属性数据
#     selected_features=selected_features,  # 选定的特征
#     target_column="Thickness of LITHOLOGIES(1: sand)",
#     test_size=0.3,
#     random_state=42,
#     output_dir=output_dir,
#     threshold_zero=0.1,
#     n_splits=5,
#     verbose=True,
# )

# # 预测结果可视化部分保持不变
# # 可视化预测结果 - 散点图
# plt.figure(figsize=(14, 12))
# scatter = plt.scatter(
#     svr_analysis["prediction_results"]["X"],
#     svr_analysis["prediction_results"]["Y"],
#     c=svr_analysis["prediction_results"]["Predicted_Sand_Thickness"],
#     cmap="viridis",
#     s=2,  # 点大小较小，因为点数可能很多
#     alpha=0.7,
#     vmin=0,
#     vmax=min(
#         svr_analysis["prediction_results"]["Predicted_Sand_Thickness"].max(),
#         svr_analysis["prediction_results"]["Predicted_Sand_Thickness"].mean() * 3,
#     ),  # 限制颜色范围
# )
# plt.colorbar(scatter, label="预测砂厚")
# plt.title("H6-2层砂厚预测结果空间分布")
# plt.xlabel("X坐标")
# plt.ylabel("Y坐标")
# plt.savefig(
#     os.path.join(output_dir, "H6-2_predicted_sand_thickness_map.png"),
#     dpi=300,
#     bbox_inches="tight",
# )
# plt.show()

### 结论：训练和预测的数据，尺度不一致！

上个月用的是尺度不一致的数据！训练数据（井点数据）和预测数据（地震属性数据）来自不同的数据处理流程或使用了不同的单位。在机器学习中，这种情况会导致模型在新数据上彻底失效。
